In [6]:
import pandas as pd
from scipy.stats import norm
import numpy as np
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (15, 10)

In [7]:
chain = pd.read_csv("option_chain.csv")

In [14]:
def calc_values(chain):
    underlying_price = 4106.62
    dte = 5
    risk_free_rate = 0.0006
    atm_contract_index = (np.abs(chain["strike"] - 4106.62)).idxmin()
    atm_impliedvol = chain.iloc[atm_contract_index]["impvol"]

    # Calculate option value for all options using ATM volatility
    chain["values"] = chain.apply(
        lambda x: bs_price(
            x["right"],
            x["underprice"],
            x["strike"],
            dte / 252,
            atm_impliedvol,
            risk_free_rate,
        ),
        axis=1,
    )
    chain["mid_price"] = (chain["bid"] + chain["ask"]) / 2
    chain["skew_premium"] = chain["mid_price"] - chain["values"]

    return chain

In [11]:
def bs_price(right, S, K, T, sigma, r):
    """
    Return's option price via Black-Scholes
    
    right: "P" or "C"
    S: Underlying price
    K: Strike price
    T: time to expiration (in days)
    sigma: volatility of the underlying
    r: interest rate (in daily terms)
    """
    d1 = (1 / (sigma * np.sqrt(T))) * (np.log(S / K) + (r + sigma**2 / 2) * T)
    d2 = d1 - sigma * np.sqrt(T)
    
    if right == "C":
        price = norm.cdf(d1) * S - norm.cdf(d2) * K * np.exp(-r * T)
        return price
    
    if right == "P":
        price = norm.cdf(-d2) * K * np.exp(-r * T) - norm.cdf(-d1) * S
        return price

In [6]:
underlying_price = 4106.62
dte = 5
risk_free_rate = .0006
atm_contract_index = (np.abs(chain["strike"] - 4106.62)).idxmin()
atm_impliedvol = chain.iloc[atm_contract_index]["impvol"]

# Calculate option value for all options using ATM volatility
chain["values"] = chain.apply(lambda x: bs_price(x["right"], underlying_price, x["strike"], dte / 252,
                                                 atm_impliedvol, risk_free_rate), axis=1)
chain["mid_price"] = (chain["bid"] + chain["ask"]) / 2
chain["skew_premium"] = chain["mid_price"] - chain["values"]

In [195]:
put_contract = chain[chain["right"] == "P"]["skew_premium"].idxmax()
put_contract = chain.iloc[put_contract]
# Buy put option so our margin required is $1000
put_contract_hedge = chain[(chain["strike"] == (put_contract["strike"] - 10)) & (chain["right"] == "P")].squeeze()

# Find the corresponding call option to make the position delta neutral
put_contract_delta = put_contract["delta"]
call_contract = np.abs(chain[chain["right"] == "C"]["delta"] + put_contract_delta).idxmin()
call_contract = chain[chain["right"] == "C"].iloc[call_contract]
# Find respective call hedge option
call_contract_hedge = chain[(chain["strike"] == (call_contract["strike"] + 10)) & (chain["right"] == "C")].squeeze()

In [241]:
class ShortIronCondor():
    def __init__(self, put, call, hedge_put, hedge_call):
        self.put = put
        self.call = call
        self.hedge_put = hedge_put
        self.hedge_call = hedge_call
        self.premium = (call["mid_price"] + put["mid_price"]) - (hedge_put["mid_price"] + hedge_call["mid_price"])
        self.max_loss = put["strike"] - hedge_put["strike"] - self.premium
        
    def pnl(self, underlying_price):
        if self.call["strike"] > underlying_price > self.put["strike"]:
            pnl = self.premium
        elif underlying_price < self.put["strike"]:
            pnl = max((underlying_price - self.put["strike"]) + self.premium, -self.max_loss)
        elif underlying_price > self.call["strike"]:
            pnl = max((self.call["strike"] - underlying_price) + self.premium, -self.max_loss)
        return pnl

In [293]:
vols = norm.rvs(.10, .01, 5000)
returns = norm.rvs(0, vols / np.sqrt(52))
prices = underlying_price * (1 + returns)

position = ShortIronCondor(put_contract, call_contract, put_contract_hedge, call_contract_hedge)
vfunc = np.vectorize(position.pnl)
pnl = vfunc(prices)

In [304]:
profit_percent = np.sum(pnl > 0) / len(pnl)
mean_profit = np.mean(pnl[pnl > 0]) / 10 
mean_loss = -np.mean(pnl[pnl < 0]) / 10

In [306]:
(profit_percent / mean_loss) - ((1-profit_percent) / mean_profit)

0.3685352056030893